In [76]:
# import finplot as fplt
import numpy as np
import pandas as pd
import pandas_ta as ta
import vectorbt as vbt
import matplotlib.pyplot as plt
import mplfinance as fplt
# import pandas_bokeh
from binance.client import Client
# %matplotlib widget
%matplotlib qt

In [77]:

client = Client()


symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT', 'LTCUSDT', "ADAUSDT", "AXSUSDT", "DOTUSDT"]
interval = "30m"

# symbol = "BNBUSDT"
# symbol="AXSUSDT"
# symbol= "DOTUSDT"
# symbol= "LINKUSDT"
# symbol = "BTCUSDT"
# symbol = "ADAUSDT"
symbol = "ETHUSDT"
# symbol = "MANAUSDT"
# symbol = "SANDUSDT"
# %%

# %%
klines = client.get_historical_klines(symbol, interval, "10 day ago")
# klines = client.get_historical_klines("BNBUSDT", interval, "3 day ago")
# klines = client.get_historical_klines("BTCUSDT", interval, "3 day ago")
#klines = client.get_historical_klines("ADAUSDT", interval, "3 day ago")
# klines = client.get_historical_klines("ETHUSDT", interval, "3 day ago")



def to_datetime_tz(arg, timedelta=-pd.Timedelta("03:00:00"), unit="s", **kwargs):
    """
    to_datetime_tz(arg, timedelta=-pd.Timedelta("03:00:00"), unit="s", **kwargs)
    Args:
        arg (float): epochtime
        timedelta (pd.Timedelta): timezone correction
        unit (string): unit in which `arg` is
        **kwargs: pd.to_datetime remaining kwargs
    Returns:
    pd.Timestamp: a timestamp corrected by the given timedelta
    """
    ts = pd.to_datetime(arg, unit=unit)
    return ts + timedelta


def trim_data(klines):

    df = pd.DataFrame(data=klines)
    DOHLCV = df.iloc[:, [0, 1, 2, 3, 4, 5]]
    dates = to_datetime_tz(DOHLCV[0], timedelta =pd.Timedelta("00:00:00"), unit="ms")
    OHLCV = DOHLCV.iloc[:, [1, 2, 3, 4, 5]].astype("float64")

    DOHLCV = pd.concat([dates, OHLCV], axis=1)
    DOHLCV.columns = ["date", "open", "high", "low", "close", "volume"]
    DOHLCV = DOHLCV.set_index("date")
    return DOHLCV

In [78]:
kl = trim_data(klines)

In [79]:
w1 = 5

phi = lambda i: 0.618 ** i
phis = [phi(i/2) for i in range(5)]; phis.reverse()
l = [1 + phi(i/2) for i in range(9)]; l.reverse()
phis += l
l = [1 + el for el in l]
phis += l
phis
# phi(4)+phi(0)

[0.381924,
 0.4858281918538693,
 0.618,
 0.7861297602813418,
 1.0,
 1.145865941776,
 1.1855494463455971,
 1.236029032,
 1.3002418225656913,
 1.381924,
 1.4858281918538694,
 1.6179999999999999,
 1.7861297602813417,
 2.0,
 2.145865941776,
 2.185549446345597,
 2.2360290320000002,
 2.3002418225656913,
 2.3819239999999997,
 2.4858281918538694,
 2.618,
 2.7861297602813417,
 3.0]

In [80]:
# aberr = kl.ta.aberration()
# d = aberr[["ABER_ATR_5_15"]]
# aberr.drop("ABER_ATR_5_15", axis=1, inplace=True)
klmeans = kl.ewm(span=w1).mean()
klstds = kl.ewm(span=w1).std()
# havg = kl.high.ewm(span=3)

In [81]:
klmeans

,open,high,low,close,volume
date,,,,,
2021-11-25 20:30:00,4521.660000,4528.980000,4505.710000,4510.010000,5531.109300
2021-11-25 21:00:00,4514.664000,4527.192000,4502.014000,4514.240000,5031.472080
2021-11-25 21:30:00,4515.813158,4523.984211,4495.849474,4506.661053,5167.188679
2021-11-25 22:00:00,4508.509385,4515.006000,4484.426615,4500.695692,5539.854318
2021-11-25 22:30:00,4502.286825,4515.948057,4487.191896,4506.981564,4623.920822
...,...,...,...,...,...
2021-12-05 18:00:00,4131.261888,4154.875154,4112.471073,4134.509280,12386.748195
2021-12-05 18:30:00,4134.501259,4162.583436,4121.477382,4138.452853,10256.507397
2021-12-05 19:00:00,4138.447506,4158.508958,4112.931588,4132.505236,9889.815264


In [82]:
havg = kl.high.ewm(span=w1).mean()
hstd = kl.high.ewm(span=w1).std()
lavg = kl.low.ewm(span=w1).mean()
lstd = kl.low.ewm(span=w1).std()
close_ema = kl.close.ewm(span=w1).mean()
close_std = kl.close.ewm(span=w1).std()
ohlc4 = kl.ta.ohlc4(length=w1)


In [83]:
supwidth = 100*(((kl.high - (kl.close + kl.open)/2))/(kl.high-kl.low))
infwidth = 100*((((kl.close + kl.open)/2 - kl.low))/(kl.high-kl.low))

In [84]:
csup = havg + 0.6*(hstd*2 + close_std*supwidth/100)/3
cinf = lavg - 0.6*(lstd*2 + close_std*infwidth/100)/3
cres = pd.concat([cinf, close_ema, csup], axis=1)

In [85]:
supwidth

date
2021-11-25 20:30:00    56.489042
2021-11-25 21:00:00    47.145558
2021-11-25 21:30:00    40.595162
2021-11-25 22:00:00    20.884774
2021-11-25 22:30:00    49.457995
                         ...    
2021-12-05 18:00:00    56.972955
2021-12-05 18:30:00    89.171644
2021-12-05 19:00:00    30.970286
2021-12-05 19:30:00    28.903614
2021-12-05 20:00:00    57.008885
Length: 480, dtype: float64

In [86]:

width = 100*(abs((kl.close - kl.open))/(kl.high-kl.low))

In [87]:
kl.index

DatetimeIndex(['2021-11-25 20:30:00', '2021-11-25 21:00:00',
               '2021-11-25 21:30:00', '2021-11-25 22:00:00',
               '2021-11-25 22:30:00', '2021-11-25 23:00:00',
               '2021-11-25 23:30:00', '2021-11-26 00:00:00',
               '2021-11-26 00:30:00', '2021-11-26 01:00:00',
               ...
               '2021-12-05 15:30:00', '2021-12-05 16:00:00',
               '2021-12-05 16:30:00', '2021-12-05 17:00:00',
               '2021-12-05 17:30:00', '2021-12-05 18:00:00',
               '2021-12-05 18:30:00', '2021-12-05 19:00:00',
               '2021-12-05 19:30:00', '2021-12-05 20:00:00'],
              dtype='datetime64[ns]', name='date', length=480, freq=None)

In [88]:
df = kl

# pstd2 = df.close.ewm(span=w2).std()

# def showcurves(c1):

#   fig, ax = plt.subplots()
#   ax.set_title('click on points')

#   line, = ax.plot(np.random.rand(100), 'o',
#                   picker=True, pickradius=5)  # 5 points tolerance
#   plt.show()

#   return fig, ax, lineaa


fig, axs = plt.subplots(figsize=(14, 12), nrows=2, sharex="all", gridspec_kw={"height_ratios": [3, 1]})
# fig = plt.figure(figsize=(12, 12))
# ax = plt.axes()
ax1, ax2 = axs

ax1.plot(df.close, c="w", label=f"closes", linewidth=2)
for i, phi_i in enumerate(phis):
    # csup = df.close + phi_i*(hstd + close_std*supwidth/100)/2
    # cinf = df.close - phi_i*(lstd + close_std*infwidth/100)/2
    csup = close_ema + phi_i*(hstd + close_std*supwidth/100)/2
    cinf = close_ema - phi_i*(lstd + close_std*infwidth/100)/2
    
    ax1.plot(csup, "c--", label=f"closes {w1} ema", linewidth=1.5-0.05*i)
    ax1.plot(cinf, "c--", label=f"closes {w1} ema", linewidth=1.5-0.05*i)


ax1.plot(close_ema, "y-", linewidth=2, label=f"closes {w1} ema", alpha=1)

# ax1.plot(pmean1+1.1*pstd1, "g--", label=f"closes {w1} ema", linewidth=1.5)
# ax1.plot(pmean1-1.1*pstd1, "r--", label=f"closes {w1} ema", linewidth=1.5)
# ax1.plot(pmean1+1.25*pstd1, "g--", label=f"closes {w1} ema", linewidth=1.0)
# ax1.plot(pmean1-1.25*pstd1, "r--", label=f"closes {w1} ema", linewidth=1.0)
# ax1.plot(pmean1+1.5*pstd1, "g--", label=f"closes {w1} ema", linewidth=1.0)
# ax1.plot(pmean1-1.5*pstd1, "r--", label=f"closes {w1} ema", linewidth=1.0)
# ax1.plot(pmean1+1.75*pstd1, "g--", label=f"closes {w1} ema", linewidth=1.0)
# ax1.plot(pmean1-1.75*pstd1, "r--", label=f"closes {w1} ema", linewidth=1.0)
# ax1.plot(pmean1, "y-", linewidth=2, label=f"closes {w1} ema", alpha=1)
# ax1.plot(pmean2, "y--", linewidth=2, label=f"closes {w2} ema", alpha=1)
# ax1.plot(pmean2+0.8*pstd2, "g-.", alpha=1, linewidth=1.5)
# ax1.plot(pmean2-0.8*pstd2, "r-.", alpha=1, linewidth=1.5)

macd = kl.ta.macd()
hist = macd.MACDh_12_26_9
hist_diffs = hist - hist.shift(1)
hist_colors = ["green" if c > 0 else "red" for c in hist_diffs]

hist_ema = hist.ewm(span=w1).mean()
hist_ema_diffs = hist_ema - hist_ema.shift(1)

hist_ema_colors = ["green" if c > 0 else "red" for c in hist_ema_diffs]

ax2.plot(hist, c="b", label=f"macd histogram", linewidth=1.5)
ax2.scatter(hist.index, hist, c=hist_colors, label=f"macd histogram", linewidth=0.7)
ax2.plot(hist_ema, c="y", label=f"macd histogram ema", linewidth=1.5)
ax2.scatter(hist_ema.index, hist_ema, c=hist_ema_colors, label=f"macd histogram ema", linewidth=0.7)
# ax2.plot(pmean1+1.1*pstd1, "m--", label=f"closes {w1} ema", linewidth=1.5)
# ax2.plot(pmean1-1.1*pstd1, "m--", label=f"closes {w1} ema", linewidth=1.5)
# ax2.plot(pmean1, "b-", linewidth=2, label=f"closes {w1} ema", alpha=0.8)
# ax2.plot(pmean2, "y-", linewidth=2, label=f"closes {w2} ema", alpha=0.8)
# ax2.plot(pmean2+0.8*pstd2, "g-.", alpha=0.8)
# ax2.plot(pmean2-0.8*pstd2, "r-.", alpha=0.8)
# ax2.plot(hmean2, "g--", alpha=1)
# ax2.plot(lmean2, "r--", alpha=1)
# ax2.plot(hmean3, "g-", alpha=1)
# ax2.plot(lmean3, "r-", alpha=1)
# including upper limits

yerrs = [df.high-df.close, df.low-df.close]

mom1 = df.close - df.close.shift(1)

colors = ["green" if c > 0 else "red" for c in mom1]
ax1.vlines(df.index, ymin=df.close+yerrs[1], ymax=yerrs[0]+df.close, color=colors, alpha=1)


In [89]:
fig, axs = plt.subplots(2, 1, figsize=(12, 7), sharex=True, gridspec_kw={'height_ratios': [4, 1]})

# bbplot = fplt.make_addplot(bands, ax=axs[0], linestyle="--", ylabel="BBands")
hlplot = fplt.make_addplot(cres, ax=axs[0], ylabel="Highs/Lows EMAs")
cr_width = fplt.make_addplot(width, ax=axs[1])
cs_width = fplt.make_addplot(supwidth, ax=axs[1])
ci_width = fplt.make_addplot(infwidth, ax=axs[1], ylabel="sup and infwidth")
fplt.plot(kl, type="candle", ax=axs[0], addplot=[cr_width, cs_width, ci_width, hlplot])
# fplt.plot(kl, type="candle", ax=axs[0], addplot=[bbplot, bbwidth, hlplot])


In [90]:
longs = ta.below(kl.low, close_ema - close_std*1.2).astype("bool") & (ta.above(hist, hist*0).astype("bool") & ta.increasing(hist_ema).astype("bool"))
shorts = ta.above(kl.high, close_ema + close_std*1).astype("bool")

In [91]:
 pf = vbt.Portfolio.from_signals(
     kl.close,
     entries=longs,
     size=1.,
     sl_stop=0.01, tp_stop=0.4,
     accumulate=False,
     fees=0.001)
# bt = vbt.Portfolio.from_signals(kl.close, entries=longs, short_entries=shorts, init_cash=100_000, fees=0.0006)

In [92]:
pf.benchmark_rets().sum()

-0.06814692243289033

In [93]:
longs_starts = longs.index[longs.values == True].tolist()
shorts_starts = shorts.index[shorts.values == True].tolist()

In [94]:
longs_starts[0] < longs_starts[1]

True

In [95]:
longs[longs_starts[0]], kl.high[longs_starts[0]], kl.close[longs_starts[0]], kl.low[longs_starts[0]]

(True, 4075.41, 4053.76, 4023.0)

In [96]:
l1 = longs_starts[0]
l1

Timestamp('2021-11-26 16:00:00')

In [97]:
tp = 1.5
sl = 1.2
fee = 0.1

In [98]:

hits1 = kl.high[l1:] > kl.low[l1]*(1+tp/100)
exit1 = hits1[1:].idxmax()
exit1, kl.high[exit1], kl.low[l1]

(Timestamp('2021-11-26 16:30:00'), 4103.08, 4023.0)

In [99]:

sl_hits1 = kl.low[l1:] < kl.low[l1]*(1-sl/100)
sl_exit1 = sl_hits1[1:].idxmax()

In [100]:
hits = (kl.low[l1:] < kl.low[l1]*(1-sl/100)) | (kl.high[l1:] > kl.low[l1]*(1+tp/100))
hit = hits[1:].idxmax()
hit

Timestamp('2021-11-26 16:30:00')

In [101]:
def bench_signals(close, csup, high, cinf, low, longs_starts, shorts_starts, tp=1.3, sl=1.1, fees=0.1):
    
    longs_rets = []
    shorts_rets = []
    
    searching=True
    i = 0
    ts = longs_starts[i]
    c = 0
    while searching:
        
        entry_val = cinf[ts]
        tphits = high[ts:] > entry_val*(1+tp/100) 
        slhits = low[ts:] < entry_val*(1-sl/100)
        tphit = tphits[1:].idxmax()
        slhit = slhits[1:].idxmax()
        istp = True if tphit < slhit else False

        c += 1
        if istp == True:
            exit_ts = tphit
            exit_val = entry_val*(1+tp/100)
            ret = (exit_val - entry_val - (exit_val*fee/200 + entry_val*fee/200)) / entry_val
            longs_rets.append([ts, entry_val, exit_ts, exit_val, ret])
        else:
            exit_ts = slhit
            exit_val = entry_val*(1-sl/100)
            ret = (exit_val - entry_val - (exit_val*fee/200 + entry_val*fee/200)) / entry_val
            longs_rets.append([ts, entry_val, exit_ts, exit_val, ret])
        
        for j, ts_ in enumerate(longs_starts[i:]):
            if ts_ > exit_ts:
                assert longs_starts[i+j] == ts_
                ts = ts_
                i += j
                break
        if ts == max(longs_starts) or c > len(longs_starts):
            return longs_rets
            


        
        

In [102]:
long_rets = bench_signals(kl.close, csup, kl.high, cinf, kl.low, longs_starts, shorts_starts, tp=6, sl=0.1)

In [103]:
rets_arr = np.array(long_rets)
rets_arr

array([[Timestamp('2021-11-26 16:00:00'), 3994.662235319535,
        Timestamp('2021-11-28 02:30:00'), 3990.6675730842153,
        -0.001999500000000001],
       [Timestamp('2021-11-28 09:30:00'), 4033.343937630725,
        Timestamp('2021-11-28 17:30:00'), 4029.3105936930942,
        -0.0019995000000000403],
       [Timestamp('2021-12-02 01:00:00'), 4540.653209348748,
        Timestamp('2021-12-02 01:30:00'), 4813.092401909673,
        0.05897000000000008],
       [Timestamp('2021-12-02 06:30:00'), 4474.791559126617,
        Timestamp('2021-12-02 07:00:00'), 4743.279052674215,
        0.05897000000000006],
       [Timestamp('2021-12-02 09:30:00'), 4519.53918341571,
        Timestamp('2021-12-02 10:00:00'), 4790.711534420653,
        0.058970000000000154],
       [Timestamp('2021-12-02 15:30:00'), 4479.186297107007,
        Timestamp('2021-12-02 16:00:00'), 4747.937474933427,
        0.05897000000000006],
       [Timestamp('2021-12-04 01:30:00'), 4102.9973360190725,
        Timestamp('

In [104]:
rets = rets_arr[:, 4]
rets


array([-0.001999500000000001, -0.0019995000000000403, 0.05897000000000008,
       0.05897000000000006, 0.058970000000000154, 0.05897000000000006,
       0.05897000000000015, -0.001999499999999933, -0.0019994999999999982,
       -0.001999499999999944, -0.001999499999999944,
       -0.001999499999999944, -0.001999499999999944,
       -0.001999499999999944, -0.001999499999999944], dtype=object)

In [105]:
sum(rets)

0.27485500000000096

In [73]:
retdf = pd.DataFrame(rets_arr, columns=["entry_date", "entry_value", "exit_date", "exit_value", "percentual_return"])

In [74]:
# retdf.to_csv(path_or_buf="ada_bench.csv")
# retdf.to_csv(path_or_buf="eth_bench.csv")
#retdf.to_csv(path_or_buf="btc_bench.csv")
# retdf.to_csv(path_or_buf="bnb_bench.csv")
retdf.to_csv(path_or_buf=f"{symbol}_bench.csv")